In [1]:
%tensorboard --logdir "logs/fit"

UsageError: Line magic function `%tensorboard` not found.


In [1]:
%load_ext tensorboard
import tensorflow as tf
tf.config.list_physical_devices('GPU')

print(tf.config.list_physical_devices('GPU'))

# Vincent
import numpy as np
import datetime
#import imageio
import matplotlib.pyplot as plt
from scipy import ndimage
import seaborn as sns




import os
import PIL

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


[]


In [2]:
#%cd C:\Users\emile\OneDrive - Association Cesi Viacesi mail\A5\IA\Projet
%ls

 Le volume dans le lecteur C s'appelle OS
 Le num‚ro de s‚rie du volume est B032-A377

 R‚pertoire de C:\Users\emile\OneDrive - Association Cesi Viacesi mail\A5\IA\Projet

19/12/2023  10:42    <DIR>          .
19/12/2023  10:35    <DIR>          ..
19/12/2023  10:42    <DIR>          .ipynb_checkpoints
19/12/2023  10:26             9ÿ686 liv1.ipynb
               1 fichier(s)            9ÿ686 octets
               3 R‚p(s)  11ÿ225ÿ591ÿ808 octets libres


In [4]:
train_set, val_set=tf.keras.preprocessing.image_dataset_from_directory(directory="Train", color_mode='rgb', batch_size=32, image_size=(224, 224),
    shuffle=True, subset="both", seed=10, validation_split=0.2) #subset obligatoire si c'est aleatoire, on n'a pas le dossier val séparé
test_set=tf.keras.preprocessing.image_dataset_from_directory(directory="Test", color_mode='rgb', batch_size=32, image_size=(224, 224), 
    shuffle=True, seed=10)            #(224, 224) verifier apres l'input de l'archi utilisée

Found 33136 files belonging to 5 classes.
Using 26509 files for training.
Using 6627 files for validation.
Found 8271 files belonging to 5 classes.


In [5]:

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_set.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_set.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_set.cache().prefetch(buffer_size=AUTOTUNE)

In [6]:
def learning(arch):
  model= getattr(tf.keras.applications, arch) (include_top=False ,weights='imagenet', input_shape=(224, 224, 3))
  model.trainable=False
  inputs = tf.keras.Input(shape=(224, 224, 3))
  x = model(inputs, training=True)

  x = tf.keras.layers.GlobalAveragePooling2D()(x)

  x = tf.keras.layers.Dense(512)(x) # use_bias=False  :dans certains cas, l'utilisation d'un biais peut entraîner des problèmes tels que le surapprentissage, une complexité accrue du
  # modèle et des temps de formation plus longs. Par conséquent, désactiver l'utilisation d'un biais peut aider à régulariser le modèle et améliorer sa généralisation.
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation('relu')(x)
  x = tf.keras.layers.Dense(256)(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation('relu')(x)
  x = tf.keras.layers.Dense(128)(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation('relu')(x)

  outputs = tf.keras.layers.Dense(5, activation="softmax")(x)

  new_model = tf.keras.Model(inputs, outputs)
  new_model.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

  # Using tensorboard callback
  log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_model_no_dropout"
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir=log_dir, 
  histogram_freq=1, 
  write_graph=True,
  write_images=True, 
  #write_steps_per_second=True, 
  update_freq='epoch',
  profile_batch=0, 
  embeddings_freq=0)

  checkpoint= tf.keras.callbacks.ModelCheckpoint(
  filepath=f"C:/Users/bloum/Desktop/liv1/checkpoint{arch}.h5",save_weights_only=True, monitor='val_accuracy',mode='max', save_best_only=True)
  history=new_model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=[checkpoint])
  ######################################################
  CNN_Score = new_model.evaluate(train_ds)

  print("    Train Loss: {:.5f}".format(CNN_Score[0]))
  print("Train Accuracy: {:.2f}%".format(CNN_Score[1] * 100))
  ########################################################
  CNN_Score = new_model.evaluate(val_ds)

  print("    Test Loss: {:.5f}".format(CNN_Score[0]))
  print("Test Accuracy: {:.2f}%".format(CNN_Score[1] * 100))

  loss = history.history["loss"]
  val_loss = history.history["val_loss"]

  accuracy = history.history["accuracy"]
  val_accuracy = history.history["val_accuracy"]

  epochs = range(len(history.history["loss"]))

  plt.figure(figsize=(15,5))

  #plot loss
  plt.subplot(1, 2, 1)
  plt.plot(epochs, loss, label = "training_loss")
  plt.plot(epochs, val_loss, label = "val_loss")
  plt.title("Loss")
  plt.xlabel("epochs")
  plt.legend()
  #plot accuracy
  plt.subplot(1, 2, 2)
  plt.plot(epochs, accuracy, label = "training_accuracy")
  plt.plot(epochs, val_accuracy, label = "val_accuracy")
  plt.title("Accuracy")
  plt.xlabel("epochs")
  plt.legend()


In [ ]:
learning('ResNet50')

Epoch 1/100
829/829 [==============================] - ETA: 0s - loss: 0.1566 - accuracy: 0.9470

: 

In [ ]:
learning('DenseNet201')

In [ ]:
learning('MobileNet')

In [ ]:
learning('VGG16')

In [ ]:
learning('EfficientNetB7')